In [2]:
// Data Types and Functions
public record struct Position {
    public long X;
    public long Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }

    public bool IsNextTo(Position other) {
        var xDiff = Math.Abs(other.X - this.X);
        var yDiff = Math.Abs(other.Y - this.Y);
        return Math.Max(xDiff, yDiff) == 1;
    }
}

public static HashSet<Position> ParseElfLocations(IEnumerable<string> input) {
    return input.SelectMany(
        (line, y) => line.Select(
            (c, x) => (Position: new Position { X = x, Y = y }, Character: c)
        )
    )
    .Where(item => item.Character == '#')
    .Select(item => item.Position)
    .ToHashSet();
}

public enum Direction {
    Up,
    Down,
    Left,
    Right
}

public record struct Bounds {
    public Position Minimums;
    public Position Maximums;

    public long Size() {
        var rect = Maximums - Minimums;
        return (rect.X + 1) * (rect.Y + 1);
    }
}

public static Position Delta(this Direction direction) {
    return direction switch {
        Direction.Up => new Position { X = 0, Y = -1 },
        Direction.Down => new Position { X = 0, Y = 1 },
        Direction.Left => new Position { X = -1, Y = 0 },
        Direction.Right => new Position { X = 1, Y = 0 },
    };
}

public static IEnumerable<Position> DeltaChecks(this Direction direction) {
    return direction switch {
        Direction.Up => new List<Position> {
            new Position { X = -1, Y = -1 },
            new Position { X = 0, Y = -1 },
            new Position { X = 1, Y = -1 },
        },
        Direction.Down => new List<Position> {
            new Position { X = -1, Y = 1 },
            new Position { X = 0, Y = 1 },
            new Position { X = 1, Y = 1 },
        },
        Direction.Left => new List<Position> {
            new Position { X = -1, Y = -1 },
            new Position { X = -1, Y = 0 },
            new Position { X = -1, Y = 1 },
        },
        Direction.Right => new List<Position> {
            new Position { X = 1, Y = -1 },
            new Position { X = 1, Y = 0 },
            new Position { X = 1, Y = 1 },
        },
    };
}

public record struct State {
    public Queue<Direction> DirectionQueue;
    public HashSet<Position> ElfLocations;

    public (State NextState, bool AnyElvesMoved) Step() {
        var locationsCopy = ElfLocations;
        var elvesToMove = ElfLocations
            .Where(position => locationsCopy.Any(other => position.IsNextTo(other)));
        if (!elvesToMove.Any()) {
            return (this, false);
        }

        var directionQueueCopy = new Queue<Direction>(DirectionQueue);
        var moveProposals = elvesToMove.Select(elf => {
                var directionOptions = directionQueueCopy
                    .Where(
                        direction => !direction.DeltaChecks()
                                                    .Any(delta => locationsCopy.Contains(elf + delta))
                    );

                if (!directionOptions.Any()) {
                    return (OriginalLocation: elf, ProposedLocation: (Position?) null);
                }
                return (
                    OriginalLocation: elf,
                    ProposedLocation: elf + directionOptions.First().Delta()
                );
            })
            .Where(elf => elf.ProposedLocation.HasValue)
            .Select(elf => (
                    OriginalLocation: elf.OriginalLocation,
                    ProposedLocation: elf.ProposedLocation.Value
                )
            )
            .GroupBy(elf => elf.ProposedLocation);
        
        var uniqueMoveProposals = moveProposals
            .Where(proposal => proposal.Count() == 1)
            .ToDictionary(
                proposal => proposal.Single().OriginalLocation,
                proposal => proposal.Single().ProposedLocation
            );
        
        if (!uniqueMoveProposals.Any()) {
            return (this, false);
        }

        var newLocations = locationsCopy
            .Select(location => uniqueMoveProposals.ContainsKey(location) ? uniqueMoveProposals[location] : location)
            .ToHashSet();
        
        directionQueueCopy.Enqueue(directionQueueCopy.Dequeue());
        return (
            new State {
                ElfLocations = newLocations,
                DirectionQueue = directionQueueCopy,
            },
            true
        );
    }

    public Bounds Bounds() {
        return new Bounds {
            Minimums = new Position {
                X = ElfLocations.Min(p => p.X),
                Y = ElfLocations.Min(p => p.Y),
            },
            Maximums = new Position {
                X = ElfLocations.Max(p => p.X),
                Y = ElfLocations.Max(p => p.Y),
            },
        };
    }

    public string Visualise() {
        var bounds = Bounds();
        var buffer = new StringBuilder();
        for (var y = bounds.Minimums.Y - 1; y <= bounds.Maximums.Y + 1; y++) {
            for (var x = bounds.Minimums.X - 1; x <= bounds.Maximums.X + 1; x++) {
                if (ElfLocations.Contains(new Position { X = x, Y = y})) {
                    buffer.Append("#");
                } else {
                    buffer.Append(".");
                }
            }
            buffer.Append("\n");
        }
        return buffer.ToString();
    }
}


In [3]:
// Read input
var input = System.IO.File.ReadAllLines("input.txt");
input

index,value
0,#.#######..#...#....#.##..###..#..###..#.#..###.#.#...######..#..#.#.###..
1,....#.####...##.#...#..#...#.###..#.###...#..#.##.#....#...#########...#..
2,#...#...###.####...#..###.##.##....######..###.###.##...##..#.#.#.##...###
3,##..###.#....#..#.#..###.#.####...##.#..#.#.#...#.###...#####.....####.#.#
4,.#####...#.#.#.....#.###.......#.#...#####...#..#.###.###.#.#....#####.#..
5,.#...###..#.#..#..#...####.#..###.#.####..###.#.#....#..#..#..##.##...#.#.
6,.##...#.##..#.##.#.##..##...#..#...####..#.#..##.#####.#..#....##...#....#
7,.###.#.#.#..###..#.#..##.####...#..#..#.#.########.#...#....##..##.#..###.
8,..#..#.##.##..####......#..######..####.##..#..#....######..##.#...#...###
9,.#....##.###.....#.#...#.##....#.#.#...##.#..#.##.#...##..#.#######..###..


In [4]:
var startingLocations = ParseElfLocations(input);
var startingState = new State {
    ElfLocations = startingLocations,
    DirectionQueue = new Queue<Direction>(
        new List<Direction> {
            Direction.Up,
            Direction.Down,
            Direction.Left,
            Direction.Right,
        }
    )
};

var finalState = Enumerable.Range(0, 10)
    .Aggregate(startingState, (state, _) => state.Step().NextState);
finalState.Bounds()
    .Size() - finalState.ElfLocations.Count

4138

In [5]:
var startingLocations = ParseElfLocations(input);
var state = new State {
    ElfLocations = startingLocations,
    DirectionQueue = new Queue<Direction>(
        new List<Direction> {
            Direction.Up,
            Direction.Down,
            Direction.Left,
            Direction.Right,
        }
    )
};

//startingLocations.Count()

var elvesMoved = true;
var roundNumber = 0;
while (elvesMoved && roundNumber < 100_000) {
    roundNumber += 1;
    (state, elvesMoved) = state.Step();
}
roundNumber


1010